# **Análisis con Machine Learning**
## **Taller 1**
#### **Andrea Bayona - Juan Pablo Cano**


En la actualidad, el sector inmobiliario ruso está en pleno auge. Ofrece muchas oportunidades emocionantes y un alto rendimiento en cuanto a estilos de vida e inversiones. El mercado inmobiliario lleva varios años en fase de crecimiento, lo que significa que todavía se pueden encontrar propiedades a precios muy atractivos, pero es muy probable que aumenten en el futuro. Para poder entender el mercado, una inmobiliaria rusa le ha brindado la información de la venta de más de 45 mil inmuebles entre los años de 2018 y 2021. Y quieren entender cuáles son las características principales que inciden en los precios de venta, para poder proponer planes de construcción de inmuebles en las áreas urbanas disponibles, que tomen en cuenta estas características.

In [ ]:
!shred -u setup_colab_general.py
!wget -q "https://github.com/jpcano1/python_utils/raw/main/setup_colab_general.py" -O setup_colab_general.py
import setup_colab_general as setup_general

setup_general.setup_general()

In [ ]:
!pip install --disable-pip-version-check --progress-bar off -q https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
!pip install --disable-pip-version-check --progress-bar off -q tabulate

## **Importando la librerías necesarias**

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.style.use("seaborn-deep")

# Librerías extras
import itertools
from typing import Optional

import pandas_profiling
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from tabulate import tabulate

from utils import general as gen

In [ ]:
data_url = (
    "https://raw.githubusercontent.com/"
    "Camilojaravila/202210_MINE-4206_ANAL"
    "ISIS_CON_MACHINE_LEARNING/main/Taller%"
    "201/russian_prices.csv"
)
gen.download_content(data_url, filename="russian_prices.csv")

## **Lectura y perfilamiento**

### **Diccionario de Datos**
La inmobiliaria ha construido el siguiente diccionario de datos:

* date - Fecha de publicación del anuncio.
* time - Tiempo que la publicación estuvo activo.
* geo_lat - Latitud.
* geo_lon - Longitud.
* region - Region de Rusia. Hay 85 regiones en total.
* building_type - Tipo de Fachada. 0 - Other. 1 - Panel. 2 - Monolithic. 3 - * Brick. 4 - Blocky. 5 - Wooden.
* object_type - Tipo de Apartmento. 1 - Secondary real estate market; 2 - New * building.
* level - Piso del Apartamento.
* levels - Número de pisos.
* rooms - Número de Habitaciones. Si el valor es "-1", Significa que es un "studio apartment".
* area - Área total del apartamento (m2).
* kitchen_area - Área de la Cocina (m2).
* price - Precio. En rublos

A continuación, se leen los datos y se revisan las primeras líneas para verficar que la carga fue exitosa

In [ ]:
russian_prices_df = pd.read_csv("data/russian_prices.csv")

In [ ]:
russian_prices_df.head()

In [ ]:
russian_prices_df.info()

In [ ]:
profiler = pandas_profiling.ProfileReport(russian_prices_df, dark_mode=True)

- El perfilamiento se encuentra en los anexos.

In [ ]:
if not os.path.exists("profiling_reports"):
    os.makedirs("profiling_reports")
profiler.to_file("profiling_reports/russian_prices_profile.html")

- Las siguientes columnas se eliminaron bajo el supuesto de que no son necesarias para el objetivo de negocio. La primera columna es un identificador de propiedad, ergo, no es significativa. Las columnas `time` y `date` son columnas relacionadas a la publicación, más no a la propiedad perse, por lo tanto, no son significativas para nuestro modelo.

In [ ]:
columns_to_delete = [
    "Unnamed: 0",
    "time",
    "date",
]

In [ ]:
russian_prices_df.drop(columns=columns_to_delete, inplace=True)

- Todas las columnas con valores nulos fueron removidas

In [ ]:
russian_prices_df.dropna(inplace=True)

In [ ]:
russian_prices_df.info()

In [ ]:
russian_prices_df = russian_prices_df.apply(lambda x: x.astype("int32"))
russian_prices_df["object_type"] = russian_prices_df["object_type"].apply(
    lambda x: 2 if x == 11 else x
)
russian_prices_df["rooms"] = russian_prices_df["rooms"].apply(lambda x: -1 if x == -2 else x)

In [ ]:
rows_to_drop = russian_prices_df.query(
    "kitchen_area + 5 >= area | area <= 10 | price <= 2000"
).index
russian_prices_df = russian_prices_df.drop(rows_to_drop).reset_index(drop=True)

In [ ]:
X, y = russian_prices_df.drop("price", axis=1), russian_prices_df["price"]

In [ ]:
full_X_train, X_test, full_y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234
)
X_train, X_val, y_train, y_val = train_test_split(
    full_X_train, full_y_train, test_size=0.2, random_state=1234
)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_val.shape, y_val.shape

In [ ]:
X_test.shape, y_test.shape

## **Modelamiento**

### **Regresión Polinómial**
#### **Entrenamiento**

Se define la clase para realizar la transformación polinomial de nuetras variables

In [ ]:
class ToPolynomial(BaseEstimator, TransformerMixin):
    def __init__(self, k: int = 2) -> None:
        self.k = k

    def fit(self, X, y=None):
        return self

    def transform(self, X: pd.DataFrame, y: Optional[pd.Series] = None) -> pd.DataFrame:
        columns = X.columns
        X_train_pol = pd.concat(
            [X ** (i + 1) for i in range(self.k)], axis=1
        )  # Polinomios sin interacciones
        X_train_pol.columns = np.reshape(
            [[i + " " + str(j + 1) for i in columns] for j in range(self.k)], -1
        )
        temp = pd.concat(
            [X[i[0]] * X[i[1]] for i in list(itertools.combinations(columns, 2))],
            axis=1,
        )  # Combinaciones sólo de grado 1
        temp.columns = [" ".join(i) for i in list(itertools.combinations(columns, 2))]
        X_train_pol = pd.concat([X_train_pol, temp], axis=1)
        return X_train_pol

Se crea un pipeline para encapsular los pasos de entrenamiento de nuestro modelo. Primero se realiza la transformación polinamial de las variables y estas se utilizan para entrenar el modelo de regresión lineal.

In [ ]:
estimators = [("polinomial", ToPolynomial()), ("regresion", LinearRegression())]

pipe_pol = Pipeline(estimators)

pipe_pol.fit(X_train, y_train)

 Parámetros entrenados por la Regresión Polinomial

In [ ]:
reg_lineal = pipe_pol["regresion"]

print("Intercept: ", reg_lineal.intercept_)
print("Coefficients: ", reg_lineal.coef_)

#### **Validación**

In [ ]:
y_pred = pipe_pol.predict(X_val)
y_pred

In [ ]:
r2_poly = r2_score(y_val, y_pred)
mse_poly = mean_squared_error(y_val, y_pred)
mae_poly = mean_absolute_error(y_val, y_pred)

print("------------ Polynomial Regression ------------")
print(f"R2-score: {r2_poly:.7f}")
print(f"Residual sum of squares (MSE): {mse_poly:.5f}")
print(f"Mean absolute error: {mae_poly:.5f}")

#### **Comportamiento de los datos reales vs los datos predecidos**

In [ ]:
%matplotlib inline


def draw_chart(y_val_p, y_pred_p, title, legend):
    fig, axs = plt.subplots(1, figsize=(20, 10))

    xvals = list(range(len(y_val_p[:50])))
    axs.plot(xvals, y_pred_p[:50], "bo-", label=legend)
    axs.plot(xvals, y_val_p[:50], "ro-", label="Real")

    axs.set(title=title, ylabel=y_train.name)
    axs.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
draw_chart(y_val, y_pred, "Predicción con Regresión Polinomial", "Regresión Polinomial")

### **Regresión Ridge**
#### **Entrenamiento (Sin estandarización)**



In [ ]:
ridge_reg = Ridge()
ridge_reg.fit(X_train, y_train)

In [ ]:
ridge_coef = dict(zip(X_train.columns, ridge_reg.coef_))
ridge_coef

#### **Validación**

In [ ]:
y_pred_2 = ridge_reg.predict(X_val)

In [ ]:
r2_ridge = r2_score(y_val, y_pred_2)
mse_ridge = mean_squared_error(y_val, y_pred_2)
mae_ridge = mean_absolute_error(y_val, y_pred_2)

print("------------ Ridge ------------")
print(f"R2-score: {r2_ridge:.7f}")
print(f"Residual sum of squares (MSE): {mse_ridge:.5f}")
print(f"Mean absolute error: {mae_ridge:.5f}")

#### **Comportamiento de los datos reales vs los datos predecidos**

In [ ]:
%matplotlib inline
draw_chart(y_val, y_pred_2, "Predicción con regresion Ridge", "Regresion Ridge")

#### **Entrenamiento (Con estandarización)**

In [ ]:
pipeline_ridge = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("regressor", Ridge()),
    ],
)

pipeline_ridge.fit(X_train, y_train)

In [ ]:
ridge_coef = dict(zip(X_train.columns, pipeline_ridge.steps[1][1].coef_))
ridge_coef

#### **Validación**

In [ ]:
y_pred_2b = pipeline_ridge.predict(X_val)

In [ ]:
r2_ridge_s = r2_score(y_val, y_pred_2b)
mse_ridge_s = mean_squared_error(y_val, y_pred_2b)
mae_ridge_s = mean_absolute_error(y_val, y_pred_2b)

print("------------ Ridge (Con estandarización) ------------")
print(f"R2-score: {r2_ridge_s:.7f}")
print(f"Residual sum of squares (MSE): {mse_ridge_s:.5f}")
print(f"Mean absolute error: {mae_ridge_s:.5f}")

#### **Comportamiento de los datos reales vs los datos predecidos**

In [ ]:
%matplotlib inline
draw_chart(
    y_val,
    y_pred_2b,
    "Predicción con regresion Ridge (Con estandarización)",
    "Regresion Ridge",
)

### **Regresión Lasso**
#### **Entrenamiento (Sin estandarización)**

In [ ]:
lasso_reg = Lasso()
lasso_reg.fit(X_train, y_train)

In [ ]:
lasso_coef = dict(zip(X_train.columns, lasso_reg.coef_))
lasso_coef

#### **Validación**

In [ ]:
y_pred_3 = lasso_reg.predict(X_val)

In [ ]:
r2_lasso = r2_score(y_val, y_pred_3)
mse_lasso = mean_squared_error(y_val, y_pred_3)
mae_lasso = mean_absolute_error(y_val, y_pred_3)

print("------------ Lasso ------------")
print(f"R2-score: {r2_lasso:.4f}")
print(f"Residual sum of squares (MSE): {mse_lasso:.5f}")
print(f"Mean absolute error: {mae_lasso:.5f}")

#### **Comportamiento de los datos reales vs los datos predecidos**

In [ ]:
%matplotlib inline
draw_chart(y_val, y_pred_3, "Predicción con regresion Lasso", "Regresion Lasso")

#### **Entrenamiento (Con estandarización)**

In [ ]:
pipeline = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("regressor", Lasso()),
    ],
)

pipeline.fit(X_train, y_train)

In [ ]:
lasso_coef = dict(zip(X_train.columns, pipeline.steps[1][1].coef_))
lasso_coef

#### **Validación**

In [ ]:
y_pred_3b = pipeline.predict(X_val)

In [ ]:
r2_lasso_s = r2_score(y_val, y_pred_3b)
mse_lasso_s = mean_squared_error(y_val, y_pred_3b)
mae_lasso_s = mean_absolute_error(y_val, y_pred_3b)

print("------------ Lasso (Con estandarización) ------------")
print(f"R2-score: {r2_lasso_s:.7f}")
print(f"Residual sum of squares (MSE): {mse_lasso_s:.5f}")
print(f"Mean absolute error: {mae_lasso_s:.5f}")

#### **Comportamiento de los datos reales vs los datos predecidos**

In [ ]:
%matplotlib inline
draw_chart(
    y_val,
    y_pred_3b,
    "Predicción con regresion Lasso (Con estandarización)",
    "Regresion Lasso",
)

### **Selección del mejor modelo**
Tabla comparativa con los resultados de las métricas R2, MSE y MAE para los 3 modelos entrenados.

In [ ]:
info = {
    "Model": ["Poly Regression", "Ridge", "Ridge (con S)", "Lasso", "Lasso (con S)"],
    "R2": [r2_poly, r2_ridge, r2_ridge_s, r2_lasso, r2_lasso_s],
    "MSE": [mse_poly, mse_ridge, mse_ridge_s, mse_ridge, mse_lasso_s],
    "MAE": [mae_poly, mae_ridge, mae_ridge_s, mae_lasso, mae_lasso_s],
}

print(tabulate(info, headers="keys", tablefmt="fancy_grid"))

### **Optimización de hiperparámetros para el mejor modelo**